In [36]:
#[60]==================== Grafica Dividida =================================
from google.colab import files
import pandas as pd
from bokeh.plotting import figure, output_notebook
from bokeh.models import ColumnDataSource, TableColumn, DataTable, DateFormatter
from bokeh.palettes import Category10
from bokeh.layouts import column, row
from bokeh.embed import components
from bokeh.resources import CDN
from IPython.display import display, HTML, clear_output
import datetime
import ipywidgets as widgets

#[1]==================== Grafica Dividida =================================
output_notebook()


def graficar_todos(dfs, archivos):
    clear_output(wait=True)
    herramientas=[
          "pan",
          "wheel_zoom",
          "box_zoom",
          "reset",
          "save",
          "lasso_select",
          "box_select",
          "tap",
          "zoom_in",
          "zoom_out",
          "help"
        ]
    p = figure(
      title=f"Gráfica del archivo Toga:",
      x_axis_label="Fecha",
      y_axis_label="Nivel del mar (mm)",
      x_axis_type="datetime",
      width=900,
      height=700,
      tools=','.join(herramientas),
      toolbar_location="above",
      toolbar_sticky=False,
      margin=(5, 5, 5, 5)
    )
    #[2]==================== Grafica Dividida =================================
    # Ejemplo: Cambiar color de texto en una columna con HTMLTemplateFormatter
    css_personalizado = """
    .bk-data-table {
        background-color: #ffffff !important;
        color: #000000 !important;
        font-size: 16px !important;
        font-weight: normal !important;
        border: 9px solid #ddd !important;
        border-radius: 8px !important;
        font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif !important;
    }
    .bk-header {
        background-color: #f4d03f !important;
        color: #f9f9f9 !important;
        font-weight: bold !important;
        font-size: 18px !important;
    }
    .bk-data-table .slick-row:nth-child(even) {
        background-color: #595959 !important;
    }
    .bk-data-table .slick-row:nth-child(odd) {
        background-color: #595959 !important;
    }
    /* Selección destacada */
    .bk-data-table .slick-row.selected {
        background-color: #fffb91 !important;
        color: #000000 !important;
        font-weight: 900 !important;
        box-shadow: 0 0 10px 3px rgba(255, 255, 145, 0.6) !important;
        border: 2px solid #e1d700 !important;
        outline: none !important;
        filter: none !important;
        opacity: 1 !important;
        transition: background-color 0.3s ease, box-shadow 0.3s ease, border 0.3s ease;
    }
    .bk-data-table .slick-row:hover {
        background-color: rgba(255, 235, 59, 0.3) !important;
        color: #000000 !important;
        font-weight: 700 !important;
        box-shadow: inset 0 0 10px rgba(255, 235, 59, 0.4) !important;
        outline: 1px solid rgba(255, 235, 59, 0.7) !important;
        filter: none !important;
        opacity: 1 !important;
        cursor: pointer;
        transition: background-color 0.3s ease, box-shadow 0.3s ease, outline 0.3s ease;
    }
    .bk-data-table .slick-cell {
        border-right: 1px solid rgba(200, 200, 200, 0.5) !important;
    }
    .bk-data-table .slick-cell:last-child {
        border-right: none !important;
    }

    """
    #[3]==================== Grafica Dividida =================================
    #Uso de la tabla
    colores = Category10[10]
    source2 = []
    for i, (df, archivo) in enumerate(zip(dfs, archivos)):
        source2 = ColumnDataSource(data=dict(
            x=pd.to_datetime(df['Date']),
            y=df['Promedio'],
            labels=[f"{v:.2f}" for v in df['Promedio']]

        ))
        p.scatter('x', 'y', source=source2, legend_label=archivo, line_width=2, color=colores[i % len(colores)])
        columns = [
            TableColumn(field="x", title="Fecha", formatter=DateFormatter(format="%Y-%m-%d"),width=500),
            TableColumn(field="y", title="Nivel"),
            TableColumn(field="x", title="Hora", formatter=DateFormatter(format="%H:%M:%S")),
            TableColumn(field="Archivo", title="Archivo", width=300),
        ]
        tabla = DataTable(source=source2, columns=columns, width=700, height=300,stylesheets=[css_personalizado])
        tablas.append(tabla)  # guardamos la tabla en la lista)
        print(tablas)

    p.xaxis.axis_label = "Fecha"
    p.yaxis.axis_label = "Promedio"
    p.toolbar.autohide = True
    p.toolbar.logo = None
    return p
#[4]==================== Grafica Dividida =================================
#mostramos la grafica
def actualizar_vista():
    p = graficar_todos(dfs, archivos_cargados)
    p.title.text_font_size = "20pt"
    p.xaxis.axis_label_text_font_size = "16pt"
    p.yaxis.axis_label_text_font_size = "16pt"

    layout22 = row(p,column(*tablas))
    layout_final = column(layout22)

    script, div = components(layout_final)
    recursos = CDN.render()
    html = recursos + div + script
    display(button)
    display(HTML(html))

#[5]==================== Grafica Dividida =================================
#procesamos el archivo
def on_upload_clicked(b):

    uploaded = files.upload()
    if not uploaded:
        print("No se cargaron archivos.")
        return
    global dfs, archivos_cargados, tabla_local, tablas  # <- importante

    # Ya NO hagas dfs = [] aquí, solo modifica la global si quieres reiniciar
    # Si quieres reiniciar los datos, hazlo aquí:
    dfs.clear()
    archivos_cargados.clear()
    tabla_local.clear()
    tablas.clear()

    for archivo in uploaded.keys():
        matriz_toga = pd.read_csv(archivo, sep=r'\s+', names=["StationID", "StationName", "Date",
                            "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"],
                            engine='python', skiprows=1, na_values="9999")

        lista_fecha = []
        lista_datos = []

        for filas_dat in matriz_toga.index:
            fecha = str(matriz_toga["Date"][filas_dat])
            if fecha[8:9] == "1":
                fecha_inicial = datetime.datetime(int(fecha[:4]), int(fecha[4:6]), int(fecha[6:8]))
            else:
                fecha_inicial = datetime.datetime(int(fecha[:4]), int(fecha[4:6]), int(fecha[6:8]), 12)

            for dat in ["D1", "D2", "D3", "D4",  "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"]:
                lista_fecha.append(fecha_inicial)
                lista_datos.append(matriz_toga[dat][filas_dat])
                fecha_inicial += datetime.timedelta(hours=1)

        df = pd.DataFrame({"Date": lista_fecha, "Promedio": lista_datos,"Archivo":archivo})
        dfs.append(df)
        archivos_cargados.append(archivo)

    actualizar_vista()
#[6]==================== Grafica Dividida =================================
#botones para iniciar con la grafica
button = widgets.Button(description="Seleccionar archivos para cargar")
button.on_click(on_upload_clicked)

display(button)

[DataTable(id='p5485', ...)]
[DataTable(id='p5485', ...), DataTable(id='p5518', ...)]


Button(description='Seleccionar archivos para cargar', style=ButtonStyle())